The file `kargerMinCut.txt` contains the adjacency list representation of a simple undirected graph. There are 200 vertices labeled 1 to 200. The first column in the file represents the vertex label, and the particular row (other entries except the first column) tells all the vertices that the vertex is adjacent to. So for example, the 6$^{th}$ 
th row looks like : `6	155	56	52	120	...`. This just means that the vertex with label 6 is adjacent to (i.e., shares an edge with) the vertices with labels 155, 56, 52, 120, ... , etc.

Your task is to code up and run the randomized contraction algorithm for the min cut problem and use it on the above graph to compute the min cut. (HINT: Note that you'll have to figure out an implementation of edge contractions. Initially, you might want to do this naively, creating a new graph from the old every time there's an edge contraction. But you should also think about more efficient implementations.) (WARNING: As per the video lectures, please make sure to run the algorithm many times with different random seeds, and remember the smallest cut that you ever find.) 

In [ ]:
# Randomized Contraction (N-2 iterations)

# While there are more than 2 vertices:
#   - pick a remaining edge (u, v) uniformly at random
#   - merge (or "contract") u and v into a single vertex
#   - remove self-loops
# return cut represented by final 2 vertices

In [1]:
import numpy as np

In [2]:
G = {}

with open('kargerMinCut.txt') as f:
    lines = f.readlines()
    for line in lines:
        tmp = np.fromstring(line, dtype=int, sep='\n').tolist()
        G[ tmp[0] ] =  tmp[1:]

In [3]:
# save edges and nodes to lists 
def edges_nodes(G):
    list_nodes_seen = []
    edges = []
    for key, val in G.items():
        
        list_nodes_seen.append(key)
        
        # for each key append edges between two nodes
        for i, e in enumerate(val):
            if e not in list_nodes_seen:
                edges.append((key, e))
                
    return edges, list_nodes_seen

In [4]:
def randomized_contraction(G):
    
    # nodes in the graph
    N = len(G)
    
    # list of edges
    edges, all_nodes = edges_nodes(G)
    
    # while there are more than 2 nodes remaining
    while len(all_nodes) > 2:
              
        # randomly pick one edge
        i = np.random.randint(len(edges), dtype='int')
        
        # contract that edge and return new G without that edge
        d = edges.pop(i)
        
        # modify edges and all_nodes to have a supernode        
        newG = {}
    
        # merge two rows of G into one contracted array    
        contracted_array = []
        contracted_array.append((d[0], d[1]))
        
        for a in G[d[0]]: 
            if a not in d:
                contracted_array.append(a)
        for b in G[d[1]]:
            if b not in d:
                contracted_array.append(b)
      
        for key, val in G.items():
            if key == d[0]:
                newG[key] = contracted_array
            elif key == d[1]:
                continue
                
            else: # for the rest of keys not d[0] or d[1]
                
                if (d[0] in val) or (d[1] in val):
                    
                    tmp = []
                    for aa in val:
                        if aa == d[0] or aa == d[1]:
                            tmp.append(d)
                        else:
                            tmp.append(aa)
                    newG[key] = tmp
                    
                else:
                    newG[key] = val                
       
        # relabel nodes such that there are no tuples
        for key, val in newG.items():
            for ii, aa in enumerate(val): 
                # convert tuple supernode into int label
                if type(aa) == tuple:
                    newG[key][ii] = d[0]
                                        
        
        edges, all_nodes = edges_nodes(newG)
        
        # delete self loops
        for key, val in newG.items():
            for ii, aa in enumerate(val): 
                if key == aa:
                    val.pop(ii)

        G = newG
        
    # return the min cut represented by last two remaining vertices
    return len(edges)
    

In [5]:
%%time
# loop over random seeds, output the mincut from a lot of random runs
mincut = np.inf
for i in range(1000):
    np.random.seed(seed=i)
    tmp = randomized_contraction(G)
    if tmp < mincut:
        mincut = tmp

print('min cut: ', mincut)

min cut:  17
CPU times: user 10min 48s, sys: 12.4 s, total: 11min 1s
Wall time: 11min 17s
